In [1324]:
import nltk, string, random, re, pickle
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1360]:
datafake = pd.read_csv("/content/drive/My Drive/data/politifact_fake - politifact_fake.csv")
datareal = pd.read_csv("/content/drive/My Drive/data/politifact_real - politifact_real.csv")
datafake.drop(columns=['id','news_url'],inplace=True)
datareal.drop(columns=['id','news_url'],inplace=True)

ParserError: ignored

In [0]:
ps = PorterStemmer()
lem = WordNetLemmatizer()
stopword = set(stopwords.words( 'english' ))

In [0]:
def clean(data):
  return re.sub(r'[^(a-zA-Z)\s]','', data)
def remstop(data):
  global stopword
  return [w for w in data if w not in stopword]
def tagpos(data):
  return nltk.pos_tag(data)

In [0]:
documents=[]

def addindocs(data,flag):
  global documents
  for p in data.title:
    documents.append((p.lower(),flag))

addindocs(datareal, "true")
addindocs(datafake,"false")
random.shuffle(documents)

In [0]:
words=[]
allowedpos=["IN","NN" ,"NNP","PRP","PRP$" , "RP","VBP"]

def addinwords(data):
  global allowedpos, words
  for p in data.title:
    tokenized = word_tokenize(clean(p))
    stopremoved=remstop(tokenized)
    postagged = nltk.pos_tag(tokenized)
    for w in postagged:
      if w[1] in allowedpos:
        words.append(lem.lemmatize(w[0].lower()))

addinwords(data_real)
addinwords(data_fake)

In [0]:
words = nltk.FreqDist(words)

In [0]:
wordfeatures=[]
for word in words:
  if(words[word]>2):
    wordfeatures.append(word)

def docfeatures(doc):
    docwords = word_tokenize(doc)
    features = {}
    for word in wordfeatures:
        features[word] = (lem.lemmatize(word) in docwords)
    return features

In [0]:
featuresets = [(docfeatures(d), c) for (d,c) in documents]
trainset, testset = featuresets[:800], featuresets[800:]

In [1348]:
classifier = nltk.NaiveBayesClassifier.train(trainset)
print(nltk.classify.accuracy(classifier, testset)*100)

87.71929824561403


In [1317]:
classifier.show_most_informative_features(20)

Most Informative Features
              transcript = True             true : false  =     20.7 : 1.0
                   after = True            false : true   =     16.0 : 1.0
                    that = True            false : true   =     10.1 : 1.0
                    week = True             true : false  =      9.9 : 1.0
                    will = True            false : true   =      8.8 : 1.0
            presidential = True             true : false  =      8.5 : 1.0
                 million = True            false : true   =      8.2 : 1.0
                  before = True            false : true   =      8.2 : 1.0
                    care = True             true : false  =      7.8 : 1.0
                  health = True             true : false  =      7.8 : 1.0
                      if = True            false : true   =      7.5 : 1.0
                     you = True            false : true   =      7.3 : 1.0
                      is = True            false : true   =      7.2 : 1.0

In [0]:
currclassifier = open( "/content/drive/My Drive/data/naivebayes.pickle", "wb" )
pickle.dump (classifier, currclassifier)
currclassifier.close ()